<a href="https://colab.research.google.com/github/vasist1987/Techgig-timesInternet-hackathon/blob/main/Techgig_timesInternet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics
!pip install --quiet optuna
import optuna
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class dataImports():
    
    def __init__(self, df=None):
        self.df = df
        
    def openZipFileCsv(self,zipPath,fileName):
        zp=zipfile.ZipFile(zipPath,'r')
        self.df= pd.read_csv(zp.open(fileName))
        return self
        
    def import_Data_File(self,objectName,src_type,sheet=None):   
        if src_type=='csv':
            self.df=pd.read_csv(objectName)       
        if src_type=='excel':
            self.df=pd.read_excel(objectName, sheet_name = None)
        return self
    def import_Data_Database(self,src_type,tbname,server_name,db_name,uname,pwd):
        if src_type=='mssql':
            con=getConnectionMSSQL(server_name,db_name,uname,pwd)
            self.df=selectFromDb(tbname,server_name,db_name,uname,pwd,con)
        return self

In [4]:
df_train=(dataImports().import_Data_File('/content/drive/MyDrive/TechGig_Times_internet_Challenge/Training Data.csv','csv')).df
df_test=(dataImports().import_Data_File('/content/drive/MyDrive/TechGig_Times_internet_Challenge/Test Data.csv','csv')).df
click_log=(dataImports().import_Data_File('/content/drive/MyDrive/TechGig_Times_internet_Challenge/click_log.csv','csv')).df


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning:

Columns (22,24,25,26,29,30,31,32,33,34,46,47,48,49,50,51,52,53,54) have mixed types.Specify dtype option on import or set low_memory=False.



In [5]:
df_train.head(3)

,clientid_cr,clmbuserid_cr,conversiontime_cr,imprid_cr,adslotdimid_cr,algo_cr,audiences_cr,clickbid_cr,geodimid_cr,ip_cr,itemcolumbiaid_cr,itemid_cr,position_cr,pubclientid_cr,refurl_cr,siteId_cr,templateid_cr,goalid_cr,time_cr,adLogType_cr,v_cr,allAudiences_cr,pricingtype_cr,osId_cr,browserId_cr,cityId_cr,stateId_cr,modelDimId_cr,lookUpFrom_cr,connTypeDimId_cr,ispDimId_cr,countryDimId_cr,goalTypeId_cr,conversionDurationInMillis_cr,impressionTimeInMillis_cr,clickTimeInMillis_cr,osVerDimId_cr,uuidSource_cr,geoGrpDimId_cr,stateGrpDimId_cr,deviceId_cr,uvh_cr,uv_cr,platformId_cr,sdkVersion_cr,usrClusterId_cr,cityGrpDimId_cr,siteClusterIds_cr,refClusterId_cr,paid_cr,spend_cr,attributionType_cr,conversionid_cr,optimize_on_cr,bundleId_cr,conversion_fraud
0,75694,31629cd0-0b34-460f-aa61-80c3234e1225-1sjfw,1.620000e+12,01649076-c77f-4d70-afc8-213535427147-1sjd4,321300,38.0,"8gw,o3b",2.869316,144,XXX.XXX.XXX.XXX,5488088,49504155,1.0,2320,https%3A%2F%2Fwww.timesjobs.com%2Fcandidate%2F...,34,4140.0,6384,1.620000e+12,2,1.0,"7dn,6hs,35g,ome,omf,l32,nqo,2ws,qqo,lo4,oos,2w...",16,196058.0,129639.0,436.0,1806.0,202038.0,2,264496,280429.0,144.0,10,2332.0,1.620000e+12,1.620000e+12,199452.0,1.0,348011.0,348022.0,NaN,NaN,NaN,NaN,NaN,333495,348007.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,75694,4c98f114-c34c-48f4-84d6-50fc8e9b6d65-10wgg,1.620000e+12,f84d83b9-b757-4f97-9813-34f8dcc57cc5-1sjfw,129239,38.0,"8kv,2vk,8gw,2vl",2.790000,144,XXX.XXX.XXX.XXX,5488773,49504157,1.0,15240,https%3A%2F%2Fwww.cricbuzz.com%2Flive-cricket-...,178,663.0,6384,1.620000e+12,2,1.0,"h9r,35j,7dx,592,lo4,5o2,gau,75y,k8l,2ve,k8p,8k...",16,196058.0,129639.0,10314.0,1817.0,202038.0,2,264496,136127.0,144.0,10,3201.0,1.620000e+12,1.620000e+12,199452.0,1.0,348011.0,348024.0,NaN,NaN,NaN,NaN,NaN,NaN,348010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,75694,cd92087a-54b0-4911-bc3d-1144a3c2babd-1sjlg,1.620000e+12,061edef6-3d9d-408b-884a-ad840e6930ce-10o78,321554,21.0,"8gw,o3b",2.790000,144,XXX.XXX.XXX.XXX,5488088,49504160,1.0,2320,https%3A%2F%2Fwww.timesjobs.com%2Fcandidate%2F...,34,4144.0,6384,1.620000e+12,2,1.0,"7dn,oy7,nqo,41j,2ws,lo4,oos,oou,apc,5o2,o6b,ox...",16,196058.0,129639.0,0.0,1812.0,202038.0,2,264503,172725.0,144.0,10,2821.0,1.620000e+12,1.620000e+12,199452.0,1.0,348011.0,348027.0,NaN,NaN,NaN,NaN,NaN,128491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [6]:
df_train['conversion_fraud'].value_counts()

False    834
True     131
Name: conversion_fraud, dtype: int64

In [7]:
y=df_train['conversion_fraud'].astype(int)

In [8]:
click_log.head(3)

,Unnamed: 0,imprId,clmbUserId,adSltDimId,auds,allAuds,itemid,algo,advClientId,pubClientId,itmClmbLId,tmpltId,refUrl,clickBid,clickXForwardedFor,position,paid,siteId,callIdentifier,crtd,adslotRatio,pricingType,usrClusterId,siteClusterIds,ctxCatId,uuidSource,clickIp,clickGeoId,esi,adBlockerDetectionValueId,bundleId,impressionServedTimeStamp,appver,topic,ip,geoDimId,countryDimId,geoGrpDimId,cityDimId,cityGrpDimId,stateDimId,stateGrpDimId,ispDimId,osDimId,osVerDimId,devTypeDimId,vendorDimId,modelDimId,connTypeDimId,browserDimId,refClusterId,fraud,cpa,imprUserId,optout
0,0,72efd138-0f6a-4aed-bd3f-9bdd70c0e869-1sjts,e91fcf75-a645-40f6-ab1f-411017d1d19a-631k,207251,"2vv,8k8,8gv,2vk,8gw,joa","7dm,cy9,5b2,oy7,p98,lo4,p12,p16,apc,p18,nqy,ic...",52696204,18,330,2310,5618155,5627.0,https%3A%2F%2Fnavbharattimes.indiatimes.com%2F...,13.10040,"103.212.158.109, 23.59.168.135, 124.124.201.229",0,1,16,B166b630l8021616125598649pzmfku8tlbktbqtjht65t,1.620000e+12,0.024,6.0,"333,734,333,729,333,000","128,527,333,734,333,000",205295-1,1,103.212.158.109,144.0,0.0,4,NaN,86,NaN,AdServe,122.176.114.222,144,144,348011.0,440.0,348005.0,1802.0,348021.0,136127.0,196063.0,199414.0,196047.0,196460,202038,264494,129639,NaN,NaN,NaN,NaN,NaN
1,0,210a7e16-6783-46a5-9246-c635db091490-3qpg,be37dc3a-d12b-4aa5-bd39-1e42ede9d6b9-6mhk,356952,"8gw,2vb,o3b","hkf,n5n,59j,p9q,n3p,p9t,o1o,41j,o1n,nyw,n3s,n5...",53622457,18,330,85030,5617956,5167.0,https%3A%2F%2Fwww.olx.in%2F,4.58514,"2409:4055:2e87:42e9:4dec:ae38:609e:8932, 49.44...",1,1,3290,q613U220R8891616126263941lu54rceq6xilidopi88m0k,1.620000e+12,1.000,16.0,NaN,NaN,205295-1,1,2409:4055:2e87:42e9:4dec:ae38:609e:8932,144.0,0.0,4,NaN,12,NaN,AdServe,2409:4055:2e87:42e9:4dec:ae38:609e:8932,144,144,348011.0,10292.0,348005.0,1812.0,348027.0,280429.0,196058.0,199452.0,196047.0,196460,202038,264496,129639,NaN,NaN,NaN,NaN,NaN
2,0,179d3fbb-3e10-435d-851d-f65029ef2eb4-10ntc,faceec73-d833-4bd6-b878-3bc27d2638dd-10uao,350086,"2vv,8k8,2vk,8gw","o1j,o1o,oy7,o1n,nyw,oww,lo4,p12,apc,358,5o2,o6...",53622457,38,330,2658,5617956,5657.0,https%3A%2F%2Ftimesofindia.indiatimes.com%2F,4.58514,"2405:201:c00e:4087:5838:cbcb:8571:e8c0, 49.44....",159934117,1,3,E660S185t880,1.620000e+12,1.000,16.0,333496,"333,734,128,527,333,000",205295-1,1,2405:201:c00e:4087:5838:cbcb:8571:e8c0,144.0,0.0,4,NaN,24,NaN,AdServe,2405:201:c00e:4087:5838:cbcb:8571:e8c0,144,144,348011.0,435.0,348008.0,129163.0,348023.0,280429.0,196063.0,199414.0,196047.0,196460,202038,264496,129639,NaN,NaN,NaN,NaN,NaN


In [9]:
df_train.imprid_cr.value_counts()

54c285f5-8766-404c-ae65-c63d32f47eee-23by0    20
a665250f-42bd-40b5-b371-aa932ef0ff02-1sj4s     2
e5e142cb-d049-4b77-91b4-81bdfeee39c6-1sjac     2
7eb868a5-f588-4354-b96d-ee11e8d44b96-10o78     2
bfe22adb-b12b-4443-bffa-2e1e981db2b1-1siz8     2
                                              ..
3666e9ee-b88c-4e8d-807a-d9f8c316ee14-62ys      1
ac5c5cee-6d5b-41a8-b2b1-fdacb4540eee-1slg4     1
21189802-09e7-40aa-9014-f7fbd60b4bff-62qg      1
38f04b84-e12a-48f3-83b4-944fed76334b-1sjo8     1
0f3625c2-f8d7-4d1c-9e40-b370d0eb48c9-1sk7o     1
Name: imprid_cr, Length: 934, dtype: int64

In [10]:
df_train.conversion_fraud.value_counts()

False    834
True     131
Name: conversion_fraud, dtype: int64

In [11]:
df_train.isnull().sum().sort_values(ascending = False)

uvh_cr                           965
bundleId_cr                      960
sdkVersion_cr                    958
conversionid_cr                  941
uv_cr                            936
optimize_on_cr                   925
refClusterId_cr                  899
deviceId_cr                      854
platformId_cr                    853
paid_cr                          771
spend_cr                         771
attributionType_cr               771
usrClusterId_cr                  611
siteClusterIds_cr                553
cityGrpDimId_cr                  344
clickbid_cr                      296
audiences_cr                     199
clickTimeInMillis_cr             132
refurl_cr                        125
templateid_cr                    107
stateGrpDimId_cr                 105
allAudiences_cr                   47
osVerDimId_cr                     21
osId_cr                           21
browserId_cr                      21
modelDimId_cr                     21
position_cr                       20
c

In [12]:
click_log.isnull().sum().sort_values(ascending = False)

optout                       224546
fraud                        224177
imprUserId                   193857
cpa                          191388
refClusterId                 188613
bundleId                     143999
appver                       117879
adBlockerDetectionValueId    108712
clickBid                     103006
impressionServedTimeStamp     88104
usrClusterId                  84280
esi                           81514
browserDimId                  80265
auds                          63035
connTypeDimId                 60499
siteClusterIds                56720
refUrl                        46450
cityGrpDimId                  34603
adslotRatio                   29667
modelDimId                    22615
tmpltId                       22299
pricingType                   22299
clickIp                       22299
clickXForwardedFor            22299
clickGeoId                    22299
ctxCatId                      18407
devTypeDimId                  15356
allAuds                     

In [13]:
train = pd.merge(df_train, click_log, how='left',left_on='imprid_cr',right_on='imprId')
y_train=train['conversion_fraud'].astype(int)

In [14]:
test = pd.merge(df_test, click_log, how='left',left_on='imprid_cr',right_on='imprId')
test.head(3)

,record_id,clientid_cr,clmbuserid_cr,conversiontime_cr,imprid_cr,adslotdimid_cr,algo_cr,audiences_cr,clickbid_cr,geodimid_cr,ip_cr,itemcolumbiaid_cr,itemid_cr,position_cr,pubclientid_cr,refurl_cr,siteId_cr,templateid_cr,goalid_cr,time_cr,adLogType_cr,v_cr,allAudiences_cr,pricingtype_cr,osId_cr,browserId_cr,cityId_cr,stateId_cr,modelDimId_cr,lookUpFrom_cr,connTypeDimId_cr,ispDimId_cr,countryDimId_cr,goalTypeId_cr,conversionDurationInMillis_cr,impressionTimeInMillis_cr,clickTimeInMillis_cr,osVerDimId_cr,uuidSource_cr,geoGrpDimId_cr,stateGrpDimId_cr,deviceId_cr,uvh_cr,uv_cr,platformId_cr,sdkVersion_cr,usrClusterId_cr,cityGrpDimId_cr,siteClusterIds_cr,refClusterId_cr,paid_cr,spend_cr,attributionType_cr,conversionid_cr,optimize_on_cr,bundleId_cr,Unnamed: 0,imprId,clmbUserId,adSltDimId,auds,allAuds,itemid,algo,advClientId,pubClientId,itmClmbLId,tmpltId,refUrl,clickBid,clickXForwardedFor,position,paid,siteId,callIdentifier,crtd,adslotRatio,pricingType,usrClusterId,siteClusterIds,ctxCatId,uuidSource,clickIp,clickGeoId,esi,adBlockerDetectionValueId,bundleId,impressionServedTimeStamp,appver,topic,ip,geoDimId,countryDimId,geoGrpDimId,cityDimId,cityGrpDimId,stateDimId,stateGrpDimId,ispDimId,osDimId,osVerDimId,devTypeDimId,vendorDimId,modelDimId,connTypeDimId,browserDimId,refClusterId,fraud,cpa,imprUserId,optout
0,1000,82122,1045e8b8-662e-49b3-bc4a-4af5326c8236-62kw,1620000000000,864cdaf0-cca7-4bcc-b210-a33858cf8755-1siz8,233531,38,NaN,NaN,145,XXX.XXX.XXX.XXX,5526656,53184285,3,2658,https%3A%2F%2Ftimesofindia.indiatimes.com%2Fworld,3,2946.0,8628,1620000000000,1,1,"9nj,8qu",16,196061,0,0,2442,0,1,264494,265618,145,10,369206,1620000000000,NaN,0,1,348012,NaN,196047.0,NaN,42860785.0,1.0,NaN,NaN,NaN,3.337341e+17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,864cdaf0-cca7-4bcc-b210-a33858cf8755-1siz8,1045e8b8-662e-49b3-bc4a-4af5326c8236-62kw,233531.0,NaN,9nj,15797316.0,38.0,16421.0,2658.0,5580430.0,2946.0,https%3A%2F%2Ftimesofindia.indiatimes.com%2Fworld,NaN,"3.93.4.240, 23.32.17.157, 104.124.54.84",3.0,1.0,3.0,Q470y448o140,1.620000e+12,1.054,16.0,"2494-1,2400-1,209069-1",Impression UserAgent is Mozilla/5.0 (X11; Linu...,1,145,0,4.0,NaN,330,NaN,NaN,AdServe,3.93.4.240,145,145.0,348012.0,0.0,NaN,2442.0,NaN,265618.0,196061.0,0.0,196047.0,0.0,0,264494,0,NaN,"333,734,128,527,333,000",NaN,True,1,NaN
1,1001,75694,53a9a707-967d-4a67-8e9f-e3dd55b464e8-10wm0,1620000000000,6c44c171-9445-4e6b-856e-89bfcd0be22c-1sito,356952,20,NaN,3.1,144,XXX.XXX.XXX.XXX,5488773,49504162,1,85030,https%3A%2F%2Fwww.olx.in%2F,3290,5167.0,6384,1620000000000,2,1,"oy5,oy7,mgg,oy9,pus,lo4,p12,ps3,oos,p16,oou,o1...",16,196063,129639,10278,1808,202038,2,264496,280429,144,10,2298,1620000000000,1.620000e+12,199414,1,348011,348020.0,NaN,NaN,NaN,NaN,NaN,NaN,348006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6c44c171-9445-4e6b-856e-89bfcd0be22c-1sito,53a9a707-967d-4a67-8e9f-e3dd55b464e8-10wm0,356952.0,NaN,"oy5,oy7,mgg,oy9,pus,lo4,p12,ps3,oos,p16,oou,o1...",49504162.0,20.0,75694.0,85030.0,5488773.0,5167.0,https%3A%2F%2Fwww.olx.in%2F,3.1,"2409:4042:70d:484a:d4ad:54d7:aea3:7844, 49.44....",1.0,1.0,3290.0,k415X826S13016161523453701trjtmr2w9ru45z5ielw1f,1.620000e+12,1.000,16.0,205323-1,,1,144,0,4.0,NaN,7,NaN,AdServe,2409:4042:70d:484a:d4ad:54d7:aea3:7844,144,144,348011.0,10278.0,348006.0,1808.0,348020.0,280429.0,196063.0,199414.0,196047.0,196460.0,202038.0,264496,129639,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,75694,53a9a707-967d-4a67-8e9f-e3dd55b464e8-10wm0,1620000000000,c00eae97-46c5-48b8-89bf-9d45628b9de3-1siz8,356952,18,NaN,3.1,144,XXX.XXX.XXX.XXX,5488088,49504161,1,85030,https%3A%2F%2Fwww.olx.in%2F,3290,5167.0,6384,1620000000000,2,1,"oy7,mgg,oy9,lo4,p12,ps3,oos,p16,oou,o1v,p18,5o...",16,196063,129639,10278,1808,202038,2,264496,280429,144,10,5713,1620000000000,1.620000e+12,199414,1,348011,348020.0,NaN,NaN,NaN,NaN,NaN,NaN,348006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,c00eae97-46c5-48b8-89bf-9d45628b9de3-1siz8,53a9a707-967d-4a67-8e9f-e3dd55b464e8-10wm0,356952.0,NaN,"oy7,mgg,oy9,lo4,p12,ps3,oos,p16,oou,o1v,p18,5o...",4

In [15]:
Essential_Columns=['clientid_cr','pubclientid_cr','clickIp','clmbuserid_cr','imprid_cr','siteId_cr','goalid_cr','cityId_cr','stateId_cr','countryDimId_cr','browserId_cr','adslotdimid_cr','crtd','itmClmbLId','ispDimId_cr','devTypeDimId','osVerDimId_cr']

In [16]:
train[Essential_Columns].isnull().sum().sort_values(ascending = False)

clickIp            234
devTypeDimId       216
itmClmbLId         150
crtd               150
osVerDimId_cr       21
browserId_cr        21
cityId_cr           20
stateId_cr          20
countryDimId_cr     20
ispDimId_cr         20
adslotdimid_cr       0
goalid_cr            0
siteId_cr            0
imprid_cr            0
clmbuserid_cr        0
pubclientid_cr       0
clientid_cr          0
dtype: int64

In [17]:
pct_null = train.isnull().sum() / len(train)
missing_features = pct_null[pct_null > 0.25].index
train_new=train.drop(missing_features, axis=1)
test_new=test.drop(missing_features, axis=1)
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046 entries, 0 to 1045
Data columns (total 80 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   clientid_cr                    1046 non-null   int64  
 1   clmbuserid_cr                  1046 non-null   object 
 2   conversiontime_cr              1046 non-null   float64
 3   imprid_cr                      1046 non-null   object 
 4   adslotdimid_cr                 1046 non-null   int64  
 5   algo_cr                        1026 non-null   float64
 6   audiences_cr                   816 non-null    object 
 7   geodimid_cr                    1046 non-null   int64  
 8   ip_cr                          1026 non-null   object 
 9   itemcolumbiaid_cr              1046 non-null   int64  
 10  itemid_cr                      1046 non-null   int64  
 11  position_cr                    1026 non-null   float64
 12  pubclientid_cr                 1046 non-null   i

In [18]:
train_new.isnull().sum().sort_values(ascending = False)

cityGrpDimId                     260
impressionServedTimeStamp        258
siteClusterIds                   257
refUrl                           245
adslotRatio                      236
clickIp                          234
tmpltId                          234
clickXForwardedFor               234
pricingType                      234
clickGeoId                       234
audiences_cr                     230
uuidSource                       221
stateGrpDimId                    218
devTypeDimId                     216
modelDimId                       199
allAuds                          196
vendorDimId                      185
ctxCatId                         172
stateDimId                       162
cityDimId                        161
callIdentifier                   151
Unnamed: 0                       150
adSltDimId                       150
clmbUserId                       150
imprId                           150
algo                             150
advClientId                      150
p

In [19]:
#train_new.refUrl.str[0:
train_new.refUrl.str.find('.com',0,-1)#]

0       27.0
1       26.0
2       26.0
3       27.0
4       27.0
        ... 
1041     NaN
1042    36.0
1043    33.0
1044    -1.0
1045    -1.0
Name: refUrl, Length: 1046, dtype: float64

In [20]:
train_new['refurl_cr']=train_new.refurl_cr.apply(lambda x: str(x)[0:str(x).find('cocdn.',0,-1)]).apply(lambda x: str(x)[0:str(x).find('.com',0,-1)]).apply(lambda x: str(x)[0:str(x).find('.in',0,-1)]).apply(lambda x: str(x)[str(x).find('www.',0,-1)+4:len(x)+1]).str.replace('ps%3A%2F%2F','').str.replace('m\.','').str.replace('m-','').str.replace('p%3A%2F%2F','').str.replace('p%3A%2F%2F','')
train_new['refurl_cr']=train_new.refurl_cr.apply(lambda x: str(x)[0:str(x).find('cocdn.',0,-1)]).apply(lambda x: str(x)[0:str(x).find('-',0,-1)]).apply(lambda x: str(x)[0:str(x).find('/.',0,-1)])
test_new['refurl_cr']=test_new.refurl_cr.apply(lambda x: str(x)[0:str(x).find('cocdn.',0,-1)]).apply(lambda x: str(x)[0:str(x).find('.com',0,-1)]).apply(lambda x: str(x)[0:str(x).find('.in',0,-1)]).apply(lambda x: str(x)[str(x).find('www.',0,-1)+4:len(x)+1]).str.replace('ps%3A%2F%2F','').str.replace('m\.','').str.replace('m-','').str.replace('p%3A%2F%2F','').str.replace('p%3A%2F%2F','')
test_new['refurl_cr']=test_new.refurl_cr.apply(lambda x: str(x)[0:str(x).find('cocdn.',0,-1)]).apply(lambda x: str(x)[0:str(x).find('-',0,-1)]).apply(lambda x: str(x)[0:str(x).find('/.',0,-1)])

In [21]:
train_new_nodups=train_new[['adLogType_cr',  'adslotRatio', 'adslotdimid_cr', 'algo_cr', 'allAudiences_cr',  'audiences_cr', 'browserId_cr', 'callIdentifier', 'cityDimId', 'cityGrpDimId', 'cityId_cr', 'clickGeoId', 'clickIp', 'clickTimeInMillis_cr', 'clickXForwardedFor', 'clientid_cr',  'clmbuserid_cr', 'connTypeDimId_cr', 'conversionDurationInMillis_cr', 'conversion_fraud', 'conversiontime_cr', 'countryDimId_cr', 'crtd', 'ctxCatId', 'devTypeDimId', 'geoGrpDimId_cr', 'geodimid_cr', 'goalTypeId_cr', 'goalid_cr', 'impressionServedTimeStamp', 'impressionTimeInMillis_cr', 'imprid_cr',  'ip_cr','ispDimId_cr', 'itemcolumbiaid_cr','itemid_cr','lookUpFrom_cr','modelDimId_cr', 'osDimId', 'osId_cr','osVerDimId_cr', 'paid','position_cr', 'pricingtype_cr','pubclientid_cr', 
       'refurl_cr', 'siteClusterIds', 'siteId_cr', 'stateDimId', 'stateGrpDimId_cr', 'stateId_cr', 'templateid_cr', 'time_cr', 'tmpltId', 'topic','uuidSource_cr', 'v_cr', 'vendorDimId']]

In [22]:
train_new_nodups.isnull().sum().sort_values(ascending = False)

cityGrpDimId                     260
impressionServedTimeStamp        258
siteClusterIds                   257
adslotRatio                      236
clickXForwardedFor               234
clickIp                          234
tmpltId                          234
clickGeoId                       234
audiences_cr                     230
devTypeDimId                     216
vendorDimId                      185
ctxCatId                         172
stateDimId                       162
cityDimId                        161
callIdentifier                   151
paid                             150
crtd                             150
osDimId                          150
topic                            150
stateGrpDimId_cr                 144
clickTimeInMillis_cr             135
templateid_cr                    107
allAudiences_cr                   47
browserId_cr                      21
modelDimId_cr                     21
osVerDimId_cr                     21
osId_cr                           21
i

In [23]:
train_new_nodups.quantile([0,0.25,0.5,0.75,0.90, 0.95,0.97])

,adLogType_cr,adslotRatio,adslotdimid_cr,algo_cr,browserId_cr,cityDimId,cityGrpDimId,cityId_cr,clickGeoId,clickTimeInMillis_cr,clientid_cr,conversionDurationInMillis_cr,conversion_fraud,conversiontime_cr,countryDimId_cr,crtd,devTypeDimId,geoGrpDimId_cr,geodimid_cr,goalTypeId_cr,goalid_cr,impressionTimeInMillis_cr,ispDimId_cr,itemcolumbiaid_cr,itemid_cr,lookUpFrom_cr,modelDimId_cr,osDimId,osId_cr,osVerDimId_cr,paid,position_cr,pricingtype_cr,pubclientid_cr,siteId_cr,stateDimId,stateGrpDimId_cr,stateId_cr,templateid_cr,time_cr,tmpltId,uuidSource_cr,v_cr
0.00,1.0,0.02300,129117.0,1.0,0.0,0.0,0.0,0.00,0.0,1.610000e+12,802.00,200.00,0.0,1.620000e+12,144.0,1.620000e+12,0.0,348011.0,144.0,10.0,82.0,1.610000e+12,0.0,3238328.00,20506010.00,1.0,0.00,0.0,0.0,0.0,1.0,0.0,6.0,1.0,3.0,0.00,348020.0,0.0,663.0,1.620000e+12,663.00,1.0,1.0
0.25,2.0,1.00000,233531.0,18.0,129408.0,1802.0,348020.0,0.00,4.0,1.620000e+12,66971.25,3155.75,0.0,1.620000e+12,144.0,1.620000e+12,196182.0,348011.0,144.0,10.0,6384.0,1.620000e+12,136127.0,5488773.00,49504155.00,2.0,0.00,196047.0,129311.0,199414.0,1.0,1.0,16.0,2658.0,26.0,133765.00,348021.0,1806.0,2784.5,1.620000e+12,2946.00,1.0,1.0
0.50,2.0,1.00000,324099.5,33.0,129639.0,1808.0,348023.0,438.00,4.0,1.620000e+12,75694.00,7814.00,0.0,1.620000e+12,144.0,1.620000e+12,202038.0,348011.0,144.0,10.0,6384.0,1.620000e+12,265618.0,5488773.00,49504155.00,2.0,202038.00,196047.0,196058.0,199452.0,1.0,1.0,16.0,13540.0,178.0,180933.00,348023.0,1810.0,4241.0,1.620000e+12,4243.00,1.0,1.0
0.75,2.0,1.00000,353785.0,38.0,129639.0,1818.0,348024.0,10255.00,4.0,1.620000e+12,75694.00,90866.75,0.0,1.620000e+12,144.0,1.620000e+12,202038.0,348011.0,144.0,10.0,6384.0,1.620000e+12,280429.0,5590371.25,53184066.25,2.0,202038.00,196047.0,196061.0,206070.0,1.0,3.0,16.0,16153.0,2923.0,266913.00,348024.0,1818.0,5167.0,1.620000e+12,5167.00,1.0,1.0
0.90,2.0,1.05400,356952.0,38.0,129639.0,129163.0,348027.0,10314.00,144.0,1.620000e+12,82122.00,822817.50,1.0,1.620000e+12,145.0,1.620000e+12,202038.0,348012.0,145.0,10.0,10015.5,1.620000e+12,280429.0,5614133.00,53924222.50,2.0,202038.00,264494.0,196063.0,360833.0,1.0,96055420.0,17.0,85030.0,3290.0,280429.00,348026.0,2577.0,5472.0,1.620000e+12,5451.90,1.0,1.0
0.95,2.0,1.24500,356952.0,39.0,129639.0,129311.0,360833.0,34714.00,145.0,1.620000e+12,85572.00,41475293.75,1.0,1.620000e+12,145.0,1.620000e+12,205597.1,348012.0,145.0,10.0,10397.0,1.620000e+12,280429.0,5615449.75,53930059.00,2.0,203206.00,264496.0,196063.0,360833.0,1.0,181569932.0,17.0,85030.0,3296.0,286137.75,348027.0,129163.0,5534.5,1.620000e+12,5512.75,3.0,1.0
0.97,2.0,1.63483,356952.0,39.0,129639.0,129311.0,360833.0,34726.25,145.0,1.620000e+12,85572.00,64069703.00,1.0,1.620000e+12,145.0,1.620000e+12,264494.0,348012.0,145.0,10.0,10549.0,1.620000e+12,280429.0,5617942.00,53930337.00,2.0,204630.12,359919.3,196063.0,360849.0,1.0,181580893.5,17.0,85030.0,3296.0,348021.00,348027.0,129163.0,5605.0,1.620000e+12,5609.00,4.0,1.0


In [24]:
train_new_nodups.drop(columns='conversion_fraud', axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
train_new_nodups.drop(columns=['allAudiences_cr','audiences_cr','clickXForwardedFor','ip_cr','siteClusterIds','clmbuserid_cr','callIdentifier','imprid_cr','ctxCatId','impressionServedTimeStamp','clickIp','topic','adLogType_cr','conversionDurationInMillis_cr','clickTimeInMillis_cr','conversiontime_cr','crtd','goalTypeId_cr','impressionTimeInMillis_cr'], axis=1,inplace=True)
test_new_nodups=test_new[train_new_nodups.columns]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
train_new_nodups.head(3)

,adslotRatio,adslotdimid_cr,algo_cr,browserId_cr,cityDimId,cityGrpDimId,cityId_cr,clickGeoId,clientid_cr,connTypeDimId_cr,countryDimId_cr,devTypeDimId,geoGrpDimId_cr,geodimid_cr,goalid_cr,ispDimId_cr,itemcolumbiaid_cr,itemid_cr,lookUpFrom_cr,modelDimId_cr,osDimId,osId_cr,osVerDimId_cr,paid,position_cr,pricingtype_cr,pubclientid_cr,refurl_cr,siteId_cr,stateDimId,stateGrpDimId_cr,stateId_cr,templateid_cr,time_cr,tmpltId,uuidSource_cr,v_cr,vendorDimId
0,0.713,321300,38.0,129639.0,1806.0,348022.0,436.0,4.0,75694,264496,144.0,202038.0,348011.0,144,6384,280429.0,5488088,49504155,2,202038.0,196047.0,196058.0,199452.0,1.0,1.0,16,2320,times,34,280429.0,348022.0,1806.0,4140.0,1.620000e+12,4140.0,1.0,1.0,264496
1,1.000,129239,38.0,129639.0,1817.0,348024.0,10314.0,4.0,75694,264496,144.0,202038.0,348011.0,144,6384,136127.0,5488773,49504157,2,202038.0,196047.0,196058.0,199452.0,1.0,1.0,16,15240,cric,178,136127.0,348024.0,1817.0,663.0,1.620000e+12,663.0,1.0,1.0,264496
2,1.000,129239,38.0,129639.0,1817.0,348024.0,10314.0,4.0,75694,264496,144.0,202038.0,348011.0,144,6384,136127.0,5488773,49504157,2,202038.0,196047.0,196058.0,199452.0,1.0,1.0,16,15240,cric,178,136127.0,348024.0,1817.0,663.0,1.620000e+12,663.0,1.0,1.0,264496


In [27]:
train_new_nodups.select_dtypes(include=['object']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046 entries, 0 to 1045
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   connTypeDimId_cr  1026 non-null   object
 1   refurl_cr         1046 non-null   object
 2   vendorDimId       861 non-null    object
dtypes: object(3)
memory usage: 32.7+ KB


In [28]:

train_new_nodups.connTypeDimId_cr.value_counts()

264494             485
264496             423
264503              62
264495              43
264502               5
264,496,195,872      3
195,869,264,494      3
264499               1
264,496,195,869      1
Name: connTypeDimId_cr, dtype: int64

In [29]:
#Label Encoding for object to numeric conversion
objList = train_new_nodups.select_dtypes(include = "object").columns
from sklearn.preprocessing import LabelEncoder
lbe = LabelEncoder()
for feat in train_new_nodups.select_dtypes(include = "object").columns:
    lbe.fit(train_new_nodups[feat].astype('str').values)
    diz_map_train = dict(zip(lbe.classes_, lbe.transform(lbe.classes_)+1))

    for i in set(test_new_nodups[feat].astype('str')).difference(train_new_nodups[feat].astype('str')):
        diz_map_train[i] = 0

    train_new_nodups[feat] = [diz_map_train[i] for i in train_new_nodups[feat].astype('str').values]
    test_new_nodups[feat] = [diz_map_train[i] for i in test_new_nodups[feat].astype('str').values]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
train_new_nodups.head(3)

,adslotRatio,adslotdimid_cr,algo_cr,browserId_cr,cityDimId,cityGrpDimId,cityId_cr,clickGeoId,clientid_cr,connTypeDimId_cr,countryDimId_cr,devTypeDimId,geoGrpDimId_cr,geodimid_cr,goalid_cr,ispDimId_cr,itemcolumbiaid_cr,itemid_cr,lookUpFrom_cr,modelDimId_cr,osDimId,osId_cr,osVerDimId_cr,paid,position_cr,pricingtype_cr,pubclientid_cr,refurl_cr,siteId_cr,stateDimId,stateGrpDimId_cr,stateId_cr,templateid_cr,time_cr,tmpltId,uuidSource_cr,v_cr,vendorDimId
0,0.713,321300,38.0,129639.0,1806.0,348022.0,436.0,4.0,75694,6,144.0,202038.0,348011.0,144,6384,280429.0,5488088,49504155,2,202038.0,196047.0,196058.0,199452.0,1.0,1.0,16,2320,55,34,280429.0,348022.0,1806.0,4140.0,1.620000e+12,4140.0,1.0,1.0,35
1,1.000,129239,38.0,129639.0,1817.0,348024.0,10314.0,4.0,75694,6,144.0,202038.0,348011.0,144,6384,136127.0,5488773,49504157,2,202038.0,196047.0,196058.0,199452.0,1.0,1.0,16,15240,11,178,136127.0,348024.0,1817.0,663.0,1.620000e+12,663.0,1.0,1.0,35
2,1.000,129239,38.0,129639.0,1817.0,348024.0,10314.0,4.0,75694,6,144.0,202038.0,348011.0,144,6384,136127.0,5488773,49504157,2,202038.0,196047.0,196058.0,199452.0,1.0,1.0,16,15240,11,178,136127.0,348024.0,1817.0,663.0,1.620000e+12,663.0,1.0,1.0,35


In [31]:
import numpy as np
train_new_nodups.osDimId.value_counts()

196047.0    561
129311.0     64
264496.0     53
196061.0     37
196058.0     30
196041.0     21
0.0          19
196063.0     13
360833.0     12
199414.0     10
264494.0     10
196460.0      9
196055.0      7
196376.0      7
359921.0      7
129300.0      5
360956.0      4
264503.0      3
199452.0      3
129276.0      2
196469.0      2
196472.0      2
359919.0      2
196340.0      1
206064.0      1
196320.0      1
360849.0      1
196073.0      1
360831.0      1
360765.0      1
196471.0      1
360751.0      1
196182.0      1
196169.0      1
196042.0      1
196046.0      1
Name: osDimId, dtype: int64

In [32]:
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(train_new_nodups)

# transform training data
X_train_norm = norm.transform(train_new_nodups)

# transform testing dataabs
X_test_norm = norm.transform(test_new_nodups)

In [33]:
X_train_norm

array([[1.55091032e-01, 8.30088847e-01, 1.42280331e-03, ...,
        0.00000000e+00, 0.00000000e+00, 7.39130435e-01],
       [2.19599910e-01, 5.26950039e-04, 1.42280331e-03, ...,
        0.00000000e+00, 0.00000000e+00, 7.39130435e-01],
       [2.19599910e-01, 5.26950039e-04, 1.42280331e-03, ...,
        0.00000000e+00, 0.00000000e+00, 7.39130435e-01],
       ...,
       [           nan, 9.45797573e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 4.78260870e-01],
       [2.19599910e-01, 9.84079198e-01, 7.69082869e-04, ...,
        0.00000000e+00, 0.00000000e+00, 6.95652174e-01],
       [2.19599910e-01, 9.84079198e-01, 1.42280331e-03, ...,
        0.00000000e+00, 0.00000000e+00, 7.39130435e-01]])

In [34]:
train_x, valid_x, train_y, valid_y = train_test_split(X_train_norm, y_train, test_size=0.25,stratify=y_train)
dtrain = lgb.Dataset(train_x, label=train_y)
dvalid = lgb.Dataset(valid_x, label=valid_y)

In [35]:
class Objective:

    def __init__(self):
        self.best_booster = None
        self._booster = None

    def __call__(self, trial):
        param = {
            "objective": "binary",
            "metric": "auc",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "bagging_fraction": trial.suggest_loguniform("bagging_fraction", 0.3, 1.0),
            "feature_fraction": trial.suggest_loguniform("feature_fraction", 0.3, 1.0),
            "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 0.3, 1.0),
            "max_depth": -1,
            "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
            "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
            "num_leaves": trial.suggest_int("num_leaves", 2, 50),
            "num_iterations": trial.suggest_int("num_iterations", 20, 1000),
            "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 20, 100),
            "objective":"binary",
            "learning_rate":trial.suggest_loguniform("learning_rate", 0.01, 0.1),
            "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
            "scale_pos_weight" : train_y.size/train_y.where(train_y==0).dropna().size-1
        }

        # Add a callback for pruning.
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
        gbm = lgb.train(
            param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
        )

        self._booster = gbm

        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
        return accuracy

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_booster = self._booster

In [37]:
objective = Objective()

study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize"
)
study.optimize(objective, n_trials=10, callbacks=[objective.callback])

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_model = objective.best_booster

new_preds = best_model.predict(train_x)
new_pred_labels = np.rint(new_preds)
print(sklearn.metrics.accuracy_score(train_y, new_pred_labels))
new_preds = best_model.predict(valid_x)
new_pred_labels = np.rint(new_preds)
print(sklearn.metrics.accuracy_score(valid_y, new_pred_labels))

[I 2021-04-27 16:16:28,919] A new study created in memory with name: no-name-f77f53ca-14bd-43c8-b7de-addca92db68e
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument

[I 2021-04-27 16:16:28,973] Trial 0 finished with value: 0.7977099236641222 and parameters: {'bagging_fraction': 0.4250873577791888, 'feature_fraction': 0.6093825257029412, 'min_gain_to_split': 0.8880476535542536, 'lambda_l1': 9.422835191170499, 'lambda_l2': 0.0006838064847481803, 'num_leaves': 13, 'num_iterations': 153, 'early_stopping_rounds': 46, 'learning_rate': 0.051271571925864505, 'n_estimators': 743}. Best is trial 0 with value: 0.7977099236641222.
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead

Best trial:
  Params: 
    bagging_fraction: 0.3712064813912623
    feature_fraction: 0.5608855629068192
    min_gain_to_split: 0.4882669983484228
    lambda_l1: 3.172452347739254e-05
    lambda_l2: 0.132942106827609
    num_leaves: 2
    num_iterations: 983
    early_stopping_rounds: 99
    learning_rate: 0.03889603351194031
    n_estimators: 773
0.9017857142857143
0.8740458015267175
